In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import torch
import os
import pickle
import matplotlib.pyplot as plt

# save_path = '/content/drive/MyDrive/iam_dataset/saved_model/resnet/'
# data_path = '/content/drive/MyDrive/iam_dataset/'

data_path = '/content/drive/MyDrive/testing_proj/iam_dataset/'
save_path = '/content/drive/MyDrive/testing_proj/iam_dataset/saved_model/resnet/'

In [ ]:
# toggles
pre_trained = True
train_on_node1 = False
pre_trained_on_node1_use_on_node2 = False


save_results = False
save_model = False

In [ ]:
labelled_data = np.load(data_path + 'labelled_data.npy')

print(labelled_data.shape)

In [ ]:
labelled_data_0 = labelled_data[labelled_data[:,1] == '000']

print(labelled_data_0.shape)

if pre_trained_on_node1_use_on_node2:
  labelled_data_0 = labelled_data_0[196:(196*2)]
else:
  labelled_data_0 = labelled_data_0[:196]
print(labelled_data_0.shape)

print(np.unique(labelled_data_0[:,1]))

In [ ]:
remove_0 = labelled_data[labelled_data[:,1]!='000']

print(np.unique(remove_0[:,1]))
print(np.unique(remove_0[:,1]).shape)

In [ ]:
y = [remove_0[remove_0[:,1]==k] for k in np.unique(remove_0[:,1])]
print(len(y))

In [ ]:
labelled_data_1 = []
for i in y:
  if pre_trained_on_node1_use_on_node2:
    labelled_data_1.append(i[5:9])
  else:
    labelled_data_1.append(i[:4])


print(len(labelled_data_1))

labelled_data_1 = np.array(labelled_data_1)

print(labelled_data_1.shape)

In [ ]:
labelled_data_1=labelled_data_1.reshape(-1,2)

In [ ]:
print(np.unique(labelled_data_1[:,1]))

In [ ]:
labelled_data_new = np.concatenate((labelled_data_0, labelled_data_1))
np.random.shuffle(labelled_data_new)
print(labelled_data_new.shape) # mix of sentences from writer id 0 and others, labelled

In [ ]:
# split before augmenting to ensure the sentences are being split and not the augmented samples
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(labelled_data_new[:,0], labelled_data_new[:,1], test_size=0.33, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
from sklearn.utils import shuffle
from PIL import Image
import random

CROP_SIZE = 113
NUM_LABELS = 50

def get_augmented_sample(sample, label, sample_ratio):
    # Get current image details
    img = Image.open(sample)
    img_width = img.size[0]
    img_height = img.size[1]

    # Compute resize dimensions such that aspect ratio is maintained
    height_fac = CROP_SIZE / img_height
    size = (int(img_width * height_fac), CROP_SIZE)

    # Resize image 
    new_img = img.resize((size), Image.ANTIALIAS)
    new_img_width = new_img.size[0]
    new_img_height = new_img.size[1]

    # Generate a random number of crops of size 113x113 from the resized image
    x_coord = list(range(0, new_img_width - CROP_SIZE))
    num_crops = int(len(x_coord) * sample_ratio)
    random_x_coord = random.sample(x_coord, num_crops)
    
    # Create augmented images (cropped forms) and map them to a label (writer)
    images = []
    labels = []
    for x in random_x_coord:
        img_crop = new_img.crop((x, 0, x + CROP_SIZE, CROP_SIZE))
        # Transform image to an array of numbers
        images.append(np.asarray(img_crop))
        labels.append(label)

    return images, labels

In [ ]:
def augment(data, labels):
    augmented_sample_list = []
    augmented_label_list = []
    for i in range(len(data)):
        augmented_samples, augmented_labels = get_augmented_sample(data[i], labels[i],0.1)
        augmented_sample_list.append(augmented_samples)
        augmented_label_list.append(augmented_labels)
    return augmented_sample_list, augmented_label_list

In [ ]:
augsamps, auglbls = augment(X_train, y_train) # returns two lists of arrays
ausamps_test, auglbls_test = augment(X_test, y_test)

In [ ]:
import operator
from functools import reduce

# flattening the lists
aug_samps = reduce(operator.add, augsamps)
aug_lbls = reduce(operator.add, auglbls)

aug_samps_test = reduce(operator.add, ausamps_test)
aug_lbls_test = reduce(operator.add, auglbls_test)
print(len(aug_samps))
print(len(aug_lbls))

print(len(aug_samps_test))
print(len(aug_lbls_test))

In [ ]:
# converting lists to arrays
aug_samps_arr = np.array(aug_samps)
aug_lbls_arr = np.array(aug_lbls)
aug_samps_arr_test = np.array(aug_samps_test)
aug_lbls_arr_test = np.array(aug_lbls_test)

print(aug_samps_arr.shape)
print(aug_lbls_arr.shape)
print(aug_samps_arr_test.shape)
print(aug_lbls_arr_test.shape)

In [ ]:
X_train = aug_samps_arr
X_train = X_train.reshape(X_train.shape[0], 1, CROP_SIZE, CROP_SIZE)
X_train = X_train.astype('float16')
X_train /= 255.0
y_train = aug_lbls_arr

In [ ]:
X_test = aug_samps_arr_test
X_test = X_test.reshape(X_test.shape[0], 1, CROP_SIZE, CROP_SIZE)
X_test = X_test.astype('float16')
X_test /= 255.0
y_test = aug_lbls_arr_test

In [ ]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

In [ ]:
new = np.unique(y_train)
print(new)

y_train[y_train !='000'] = 1
new = np.unique(y_train)
print(new)

y_train[y_train=='000'] = 0 # 0 is 0
new = np.unique(y_train)
print(new)

print(y_train.shape)

y_train = y_train.astype('float16')

print(np.unique(y_train))

In [ ]:
new = np.unique(y_test)
print(new)

y_test[y_test !='000'] = 1
new = np.unique(y_test)
print(new)

y_test[y_test=='000'] = 0
new = np.unique(y_test)
print(new)

print(y_test.shape)

y_test = y_test.astype('float16')

print(np.unique(y_test))

In [ ]:
torch.manual_seed(42)
CUDA = torch.cuda.is_available()
device = torch.device('cuda' if CUDA else 'cpu')
kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}
device

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
from torch import nn


tensor_x_train = torch.Tensor(X_train).type(torch.cuda.FloatTensor) # transform to torch tensor
tensor_y_train = torch.Tensor(y_train).type(torch.cuda.FloatTensor).long()

# tensor_x_train = torch.Tensor(X_train) # transform to torch tensor
# tensor_y_train = torch.Tensor(y_train).long()

train_dataset = TensorDataset(tensor_x_train,tensor_y_train) # create your datset
train_dataloader = DataLoader(train_dataset,batch_size=64, shuffle=True, drop_last=True) # create your dataloader

In [ ]:
tensor_x_train.shape

In [ ]:
tensor_x_test = torch.Tensor(X_test).type(torch.cuda.FloatTensor) # transform to torch tensor
tensor_y_test = torch.Tensor(y_test).long().type(torch.cuda.FloatTensor).long()

# tensor_x_test = torch.Tensor(X_test) # transform to torch tensor
# tensor_y_test = torch.Tensor(y_test).long()

test_dataset = TensorDataset(tensor_x_test,tensor_y_test) # create your datset
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, drop_last=True) # create your dataloader

In [ ]:
import torch
import torchvision.models as models
if pre_trained:
  model = models.resnet18(pretrained=True)
else:
  model = models.resnet18(pretrained=False)

In [ ]:
# change first conv to take 1 channel instead of 3
print(model.conv1)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False).to(device)
print(model.conv1)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 2).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)
model.to(device)
print(next(model.parameters()).device)

In [ ]:
print(model)

In [ ]:
if pre_trained or pre_trained_on_node1_use_on_node2:
  for param in model.parameters():
    param.requires_grad = False

In [ ]:
if pre_trained or pre_trained_on_node1_use_on_node2:
  for param in model.layer4.parameters():
    param.requires_grad = True

In [ ]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2).to(device)

In [ ]:
if pre_trained_on_node1_use_on_node2:
  if os.path.exists(save_path + 'trained_on_node1.pt'):
      print('Loading checkpoint: %s' % save_path + 'trained_on_node1.pt')
      path = torch.load(save_path + 'trained_on_node1.pt')
      # epoch = path['epoch']
      model.load_state_dict(path['model'])
      optimizer.load_state_dict(path['optimizer'])

In [ ]:
for param in model.parameters():
  print(param.requires_grad)

In [ ]:
train_losses = []
train_loss_record = []

In [ ]:
from datetime import datetime
train_log_string = '%s :: Epoch %i :: Iter %i / %i :: train loss: %0.4f'
def train(epoch):
    print('Start Epoch {} Training...'.format(epoch))
    model.train()
    train_loss = []
    train_loss_sum = []
    for idx, (data, target) in enumerate(train_dataloader):
        # data = data.to(device)
        # target = target.to(device)
        # print(data.shape)
        optimizer.zero_grad()
        #  forward-pass
        output = model(data)
        loss = criterion(output, target)
        #backward-pass
        loss.backward()
        # Update the parameters
        optimizer.step()
        train_loss.append(loss.data.cpu().numpy())
        train_loss_sum.append(loss)
        if ((idx +1) % 5000) == 0:
          print(train_log_string % (datetime.now(), epoch, idx + 1, len(train_dataloader), np.mean(train_loss)))
          train_loss = []
    mean_tr = torch.mean(torch.stack(train_loss_sum))
    print('Train Loss at epoch {}: {}\n'.format(epoch, mean_tr))
    train_loss_record.append(mean_tr)
    return train_loss_record

In [ ]:
for epoch in range(10):
    tlr = train(epoch)

In [ ]:
tlr_cpu = [i.cpu().detach().numpy() for i in tlr]

plt.plot(tlr_cpu)

In [ ]:
result = []
prediction = []
model.eval()
with torch.no_grad():
    for idx, (data, target) in enumerate(test_dataloader):
        output = model(data)
        output_sm = nn.Softmax(dim=1)(output)
        _, predicted = torch.max(output, 1)
        prediction.append(predicted)
        result.append(output_sm)
        
result = torch.stack(result).detach().cpu().numpy()
prediction = torch.stack(prediction).detach().cpu().numpy()

print(result.shape, prediction.shape)

In [ ]:
print(prediction.reshape(-1).shape)

In [ ]:
print(result.reshape(-1,2).shape)

In [ ]:
# save trained model
state_dict = {
    # 'epoch': epoch,
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
}
if save_model:
  if pre_trained_on_node1_use_on_node2:
    torch.save(state_dict, save_path + 'pretrained_on_node1_reuse_node2.pt')
  elif pre_trained:
    torch.save(state_dict, save_path + 'pretrained_on_imagenet.pt')
  else:
    torch.save(state_dict, save_path + 'trained_on_node1.pt')


In [ ]:
import pandas as pd
res = pd.DataFrame(result.reshape(-1,2))
pred = pd.DataFrame(prediction.reshape(-1))

# res.to_csv(save_path + "results_pretrained_on_node1_reuse_node2.csv", header=True, index=True)
# pred.to_csv(save_path + "pred_pretrained_on_node1_reuse_node2.csv", header=True, index=True)

In [ ]:
print(res.shape, pred.shape)

y_test_new = y_train[:result.reshape(-1,2).shape[0]]

print(y_test_new.shape)

In [ ]:
result_df = pd.DataFrame({'ground truth': y_test_new, 'NN output': prediction.reshape(-1), 'probs 0': res[0], 'probs 1': res[1]})
result_df

In [ ]:
# work out the accuracy and other metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
actual = result_df['ground truth']
predicted = result_df['NN output']
# confusion matrix
matrix = confusion_matrix(actual, predicted, labels=[0,1])
print('Confusion matrix : \n',matrix)

# outcome values order in sklearn
tp, fn, fp, tn = confusion_matrix(actual, predicted, labels=[0,1]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy
matrix_2 = classification_report(actual,predicted,labels=[1,0])
print('Classification report : \n',matrix_2)

In [ ]:
import seaborn as sn
df_cm = pd.DataFrame(matrix, index = [i for i in "01"],
                  columns = [i for i in "01"])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
matrix = classification_report(actual,predicted,labels=[0,1], output_dict=True)
print('Classification report : \n',matrix)
accuracy_score(actual, predicted)

In [ ]:
matrix_df = pd.DataFrame(matrix).transpose()
if save_results:
  if pre_trained_on_node1_use_on_node2:
    matrix_df.to_csv(save_path + "classification_report_pretrained_on_node1_reuse_node2.csv", header=False, index=False)
  elif pre_trained:
    matrix_df.to_csv(save_path + "classification_report_pretrained_on_imagenet.csv", header=False, index=False)
  else:
    matrix_df.to_csv(save_path + "classification_report_trained_on_node1.csv", header=False, index=False)

# pd.to_csv("/content/drive/MyDrive/iam_dataset/saved_model/classification_report.txt",matrix)